In [ ]:
!pip install diffusers safetensors torch


In [ ]:

import torch
from diffusers import StableDiffusionXLPipeline
from safetensors import safe_open

def load_pipe_with_lora(lora_path, fp16=True):
    dtype = torch.float16 if fp16 and torch.cuda.is_available() else torch.float32
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=dtype,
        use_safetensors=True,
    )
    from diffusers.models.attention_processor import LoRAAttnProcessor, LoRAAttnProcessor2_0
    lora_cls = LoRAAttnProcessor2_0 if hasattr(torch, "compile") else LoRAAttnProcessor
    for _, module in pipe.unet.named_modules():
        if hasattr(module, "set_processor"):
            module.set_processor(lora_cls(r=4))
    tensors = {}
    with safe_open(lora_path, framework="pt", device="cpu") as f:
        for k in f.keys():
            tensors[k] = f.get_tensor(k)
    pipe.unet.load_state_dict(tensors, strict=False)
    return pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# --- Generate ---
lora_path = "./sdxl_lora_out/lora_final.safetensors"
prompt = "young male, curly hair, medium-dark skin, oval face"

pipe = load_pipe_with_lora(lora_path)
image = pipe(prompt).images[0]
image.save("suspect.png")
image